### Full integer quantization of weights and activations into 8-bit integer
---

#### Load validation data and convert

In [1]:
import data as datapy

val_data_path = "/home/jovyan/cut-data/validation/"

# load validation as representative data for quantization
print('Loading validation data...', flush=True)
x_val, y_val, labels, file_paths_val = datapy.loadData(val_data_path)
print('...Done. Loaded {} validation samples and {} labels.'.format(x_val.shape[0], y_val.shape[1]), flush=True)

# Convert to numpy arrays
import numpy as np

x_val = np.array(x_val, dtype='float32')
y_val = np.array(y_val, dtype='float32')

2023-08-05 14:48:53.664467: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-05 14:48:53.706897: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-05 14:48:53.707424: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-05 14:48:54.347320: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading validation data...
Processed 0 files. Currently processing file: /home/jovyan/cut-data/validation/non_target/0.920_0001_270097.wav
Processed 100 files. Currently processing file: /home/jovyan/cut-data/validation/non_target/0.930_0002_182583971.wav
Processed 200 files. Currently processing file: /home/jovyan/cut-data/validation/non_target/0.940_0004_534761.wav
Processed 300 files. Currently processing file: /home/jovyan/cut-data/validation/non_target/0.950_0018_226391901.wav
Processed 400 files. Currently processing file: /home/jovyan/cut-data/validation/non_target/0.970_0017_647758.wav
Processed 500 files. Currently processing file: /home/jovyan/cut-data/validation/non_target/1.000_0001_0H2uMhzSitY_520.wav
Processed 600 files. Currently processing file: /home/jovyan/cut-data/validation/non_target/1.000_0002_--ivFZu-hlc_30.wav
Processed 700 files. Currently processing file: /home/jovyan/cut-data/validation/non_target/1.000_0002_2RpOd9MJjyQ_10.wav
Processed 800 files. Currently p

##### Load dataset generator

In [13]:
import model as modelpy

batch_size = 12

val_gen = modelpy.AudioDataGenerator(file_paths_val, y_val, batch_size=batch_size)

##### Load Keras Model

In [4]:
import tensorflow as tf
import keras

# baseline keras file which still has the activation layer in the end
keras_baselineModel_activation_path = "/home/jovyan/models/checkpoints_/baseline_two_class_model_activation/"
keras_baselineModel_activation = keras.models.load_model(keras_baselineModel_activation_path)

print("Finished loading keras model")


Finished loading keras model


In [15]:
def representative_data_gen():
  for input_value, _ in val_gen:
    # Here we yield only input data because during conversion only this part is used
    yield [input_value]



converter = tf.lite.TFLiteConverter.from_keras_model(keras_baselineModel_activation)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]


tflite_quant_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpumczl62m/assets


INFO:tensorflow:Assets written to: /tmp/tmpumczl62m/assets
/opt/conda/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:887: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2023-08-05 16:34:23.913509: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-08-05 16:34:23.913563: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-08-05 16:34:23.913827: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpumczl62m
2023-08-05 16:34:23.938658: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-08-05 16:34:23.938704: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /tmp/tmpumczl62m
2023-08-05 16:34:24.019821: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-08-05 16:34:24.406891: I te

##### Evaluate

In [24]:
interpreter = tf.lite.Interpreter(model_content=tflite_quant_model)
interpreter.allocate_tensors()

tflite_size = len(tflite_quant_model) / (1024 * 1024)
print(f"Size of the TFLite model: {tflite_size:.2f} MB")

Size of the TFLite model: 6.99 MB


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


##### Save

In [27]:
# Save the quantized model:
tflite_model_quant_file = "/home/jovyan/models/checkpoints_/baseline_activation_INT8.tflite"
with open(tflite_model_quant_file, "wb") as f:
    f.write(tflite_quant_model)
print("Saved!")

Saved!
